### WORK ON THIS ###

In [1]:
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()

In [2]:
# Load Data
df_yake_claims = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake.json')

In [3]:
# Iterate over rows in dataframe
for index, row in tqdm(df_yake_claims.iterrows()):
    # Cast row 'keywords_yake' column to string
    row['keywords_yake'] = str(row['keywords_yake'])
    # Check if keywords_yake column starts with "[[[" and ends with "]]]""
    if row['keywords_yake'].startswith('[[[') and row['keywords_yake'].endswith(']]]'):
        # Remove first "[" and last "]" from keywords_yake column
        row['keywords_yake'] = row['keywords_yake'][1:-1]
    # Cast row 'keywords_yake' column to list
    row['keywords_yake'] = ast.literal_eval(row['keywords_yake'])
    # Assign modified 'keywords_yake' list to temporary variable
    keywords_yake_temp = row['keywords_yake']
    # Assign temporary variable to 'keywords_yake' column
    df_yake_claims.at[index, 'keywords_yake'] = keywords_yake_temp

515742it [01:11, 7207.88it/s]


In [4]:
df_yake_claims['keywords_yake_exploded'] = ''

# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake' column is not a list
    if not isinstance(row['keywords_yake'], list):
        continue
    # Check if 'keywords_yake' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append to 'keywords_yake_exploded' column
    else:
        for keyword in row['keywords_yake']:
            df_yake_claims.at[index, 'keywords_yake_exploded'] += keyword[0] + ', '

515742it [02:04, 4156.10it/s]


In [5]:
# Cast 'keywords_yake_exploded' column to list
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].str.split(', ')

In [6]:
# Delete last item in 'keywords_yake_exploded' cells
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].apply(lambda x: x[:-1])

# Postprocessing

In [7]:
keywords_list = []
# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Append 'keywords_yake_exploded' to keywords_list
    keywords_list += row['keywords_yake_exploded']

515742it [00:12, 41248.83it/s]


In [8]:
# Generate unique list of keywords
keywords_list_unique = list(set(keywords_list))
df_keywords_list_unique = pd.DataFrame(keywords_list_unique, columns=['keyword_yake'])

In [9]:
# Count absolute frequency of each keyword
from collections import Counter

# Step 1: Flatten the list of keywords into a single list
flattened_keywords = [keyword for keyword_list in tqdm(df_yake_claims['keywords_yake_exploded']) for keyword in keyword_list]

# Step 2: Use collections.Counter to count the occurrences of each keyword
keyword_counts = Counter(flattened_keywords)

# Step 3: Convert the keyword counts to a DataFrame
count_df = pd.DataFrame(keyword_counts.items(), columns=['keyword_yake', 'abs_frequency'])

# Step 4: Merge the count_df with df_keywords_list_unique on the 'keyword_yake' column
df_keywords_list_unique = df_keywords_list_unique.merge(count_df, on='keyword_yake', how='left')

  0%|          | 0/515742 [00:00<?, ?it/s]

100%|██████████| 515742/515742 [00:00<00:00, 2566097.02it/s]


In [10]:
### MAYBE I NEED TO REDO THIS WITH FULL TEXT CLAIMS INSTEAD OF JUST THE YAKE KEYWORDS

# Calculate document frequency by dividing absolute frequency by len of df_yake_claims
df_keywords_list_unique['doc_frequency'] = df_keywords_list_unique['abs_frequency'] / len(df_yake_claims)

In [12]:
# Prune dataframe by document frequency and absolute frequency
min_abs_frequency = 10
max_doc_frequency = 0.3

df_keywords_list_unique_pruned = df_keywords_list_unique[(df_keywords_list_unique['abs_frequency'] >= min_abs_frequency) & (df_keywords_list_unique['doc_frequency'] <= max_doc_frequency)]

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [13]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
# nltk.download('stopwords')

# Generate copy of df_keywords_list_unique_pruned
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Delete all stopwords from 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stopwords.words('english'))])
)

100%|██████████| 32749/32749 [00:03<00:00, 9834.87it/s] 


In [14]:
# Only consider n-grams with n >= 2
min_ngram_length = 2

# Generate copy of df_keywords_list_unique_pruned
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

def filter_keywords(keyword_string):
    # Split the keyword string into individual keywords
    keywords = keyword_string.split(', ')
    # Filter out keywords with fewer than 2 words
    filtered_keywords = [keyword for keyword in keywords if len(keyword.split()) >= min_ngram_length]
    # Join the filtered keywords back into a string
    return ', '.join(filtered_keywords)

# Create a copy of df_keywords_list_unique_pruned to avoid SettingWithCopyWarning
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Now use .loc to modify the 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].progress_apply(filter_keywords)

# Delete rows with empty 'keyword_yake' column
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

  0%|          | 0/32749 [00:00<?, ?it/s]

100%|██████████| 32749/32749 [00:00<00:00, 894580.53it/s]


In [17]:
### MAYBE I NEED TO REDO THIS PART WITH THE CLAIM TEXTS INSTEAD OF THE KEYWORDS

import spacy

# Download spacy model
nlp = spacy.load('en_core_web_sm')

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned.copy()

# Perform part-of-speech tagging on the 'keyword_yake' column 
# and save the POS tags in a new column 'keyword_yake_pos'
df_keywords_list_unique_pruned_pos['keyword_yake_pos'] = df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(
    lambda x: [token.pos_ for token in nlp(x)]
)

100%|██████████| 24789/24789 [00:47<00:00, 524.78it/s]


In [18]:
# Filter out all keywords that do not contain NOUN, PRON or PROPN in their POS tags
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned_pos[df_keywords_list_unique_pruned_pos['keyword_yake_pos'].progress_apply(
    lambda x: any(pos in ['NOUN', 'PRON', 'PROPN'] for pos in x)
)]

# Reset index
df_keywords_list_unique_pruned_pos.reset_index(drop=True, inplace=True)

100%|██████████| 24789/24789 [00:00<00:00, 973316.63it/s]


# Keyword Embedding

In [19]:
from sentence_transformers import SentenceTransformer
import torch

/home/thiesen/Documents/Cleantech_Concepts/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# model_climatebert = SentenceTransformer('climatebert/distilroberta-base-climate-f')
model_bertforpatents = SentenceTransformer('anferico/bert-for-patents')
# model_patentsberta = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/anferico_bert-for-patents. Creating a new one with MEAN pooling.


In [21]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available: {}".format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU available: NVIDIA RTX A4500


In [22]:
# Generate copy of df_keywords_list_unique_pruned_pos
df_keywords_list_unique_pruned_embeddings = df_keywords_list_unique_pruned_pos.copy()

# Perform sentence embedding on the 'keyword_yake' column
# df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
#     lambda x: model_climatebert.encode(x)
# )

df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
    lambda x: model_bertforpatents.encode(x)
)

# df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
#     lambda x: model_patentsberta.encode(x)
# )

100%|██████████| 23742/23742 [05:08<00:00, 76.94it/s]


# Dimensionality Reduction

In [23]:
import umap
import numpy as np
reducer = umap.UMAP(random_state=42)

In [24]:
# Stack all the embeddings into a 2D array
# embeddings_climatebert = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding'].values)
embeddings_bertforpatents = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding'].values)
# embeddings_patentsberta = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding'].values)

# Perform UMAP dimensionality reduction
# umap_embeddings_climatebert = reducer.fit_transform(embeddings_climatebert)
umap_embeddings_bertforpatents = reducer.fit_transform(embeddings_bertforpatents)
# umap_embeddings_patentsberta = reducer.fit_transform(embeddings_patentsberta)

# Assign the reduced dimension embeddings back to new DataFrame columns
# df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding_umap'] = list(umap_embeddings_climatebert)
df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding_umap'] = list(umap_embeddings_bertforpatents)
# df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding_umap'] = list(umap_embeddings_patentsberta)

/home/thiesen/Documents/Cleantech_Concepts/venv/lib64/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [25]:
# Split the UMAP coordinates into separate columns
# df_keywords_list_unique_pruned_embeddings[['climatebert_x', 'climatebert_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)
df_keywords_list_unique_pruned_embeddings[['bertforpatents_x', 'bertforpatents_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)
# df_keywords_list_unique_pruned_embeddings[['patentsberta_x', 'patentsberta_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)

# Clustering

In [26]:
# Import HDBSCAN
import hdbscan

In [27]:
# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_embeddings_hdbscan = df_keywords_list_unique_pruned_embeddings.copy()

# Perform HDBSCAN clustering on the UMAP coordinates
# clusterer_climatebert = hdbscan.HDBSCAN(min_cluster_size=250, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['climatebert_x', 'climatebert_y']])
clusterer_bertforpatents = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['bertforpatents_x', 'bertforpatents_y']])
# clusterer_patentsberta = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['patentsberta_x', 'patentsberta_y']])

# Assign the cluster labels back to the DataFrame
# df_keywords_list_unique_pruned_embeddings_hdbscan['climatebert_cluster'] = clusterer_climatebert.labels_
df_keywords_list_unique_pruned_embeddings_hdbscan['bertforpatents_cluster'] = clusterer_bertforpatents.labels_
# df_keywords_list_unique_pruned_embeddings_hdbscan['patentsberta_cluster'] = clusterer_patentsberta.labels_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [29]:
df_keywords_list_unique_pruned_embeddings_hdbscan.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_postprocessed.json', orient='records')

# Plotting

In [36]:
import plotly.express as px

In [3]:
# Filter out data points belonging to cluster -1
# df_filtered_climatebert = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['climatebert_cluster'] != -1]
df_filtered_bertforpatents = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['bertforpatents_cluster'] != -1]
# df_filtered_patentsberta = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['patentsberta_cluster'] != -1]

In [ ]:
# Plot the UMAP embeddings in three figures, color coded by cluster
fig_bertforpatents = px.scatter(
    df_filtered_bertforpatents, 
    x='bertforpatents_x', 
    y='bertforpatents_y', 
    color='bertforpatents_cluster', 
    hover_data=['keyword_yake'], 
    title='UMAP projection of the YAKE keywords (bertforpatents)',
    height=400,  # Adjust as needed
    width=600    # Adjust as needed
)
fig_bertforpatents_unfiltered = px.scatter(
    df_keywords_list_unique_pruned_embeddings_hdbscan, 
    x='bertforpatents_x', 
    y='bertforpatents_y', 
    color='bertforpatents_cluster', 
    hover_data=['keyword_yake'], 
    title='UMAP projection of the YAKE keywords (bertforpatents)',
    height=400,  # Adjust as needed
    width=600    # Adjust as needed
)
# Display the figures
fig_bertforpatents.show()
fig_bertforpatents_unfiltered.show()

# Name Cluster

In [5]:
# Build dataframe of keywords and their corresponding clusters, where each row is a cluster with list of keywords
df_keywords_clusters_bertforpatents = df_filtered_bertforpatents.groupby('bertforpatents_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')

In [6]:
from transformers import pipeline, set_seed

/home/thiesen/Documents/Cleantech_Concepts/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Initialize a text-generation pipeline with Flan-T5-large
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = pipeline('text2text-generation', model='google/flan-t5-large', device=device)
set_seed(42)

In [12]:
def generate_cluster_name(keywords):
    # Ensure the keywords are in a list format
    keywords = keywords.split(', ') if isinstance(keywords, str) else keywords
    # Select only the first 1500 keywords from the list
    selected_keywords = keywords[:1500]
    # Join the selected keywords into a string format
    keywords_str = ', '.join(selected_keywords)
    # Create a prompt from the selected keywords
    prompt = f"Come up with a specific, precise topic name, with at maximum 3 words, for the following keywords: {keywords_str}"
    # Generate a response using the GPT-3 model
    response = generator(prompt, max_length=10, do_sample=True, temperature=0.8)[0]['generated_text']
    # Extract the cluster name from the response
    cluster_name = response
    return cluster_name

# Apply the function to the 'keywords' column to generate cluster names
df_keywords_clusters_bertforpatents['cluster_name'] = df_keywords_clusters_bertforpatents['keywords'].progress_apply(generate_cluster_name)

100%|██████████| 60/60 [00:19<00:00,  3.15it/s]


# Test Data

In [3]:
df_keywords_clusters_bertforpatents = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_postprocessed_bertforpatents_clusters.json', orient='records')
df_keywords_clusters_patentsberta = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_keywords_postprocessed_patentsberta_clusters.json', orient='records')